# Video Generation Pipeline - Google Colab

This notebook handles heavy video rendering tasks using Colab's free GPU/CPU resources.

## Setup Instructions:
1. Upload your project files to Colab
2. Install dependencies
3. Upload your video data (JSON plan)
4. Run rendering
5. Download the output video


## 1. Install Dependencies


In [ ]:
# Install Node.js (v20)
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash -
!apt-get install -y nodejs

# Verify installation
!node --version
!npm --version


In [ ]:
# Install FFmpeg (required for Remotion)
!apt-get update
!apt-get install -y ffmpeg

# Verify FFmpeg
!ffmpeg -version | head -n 1


In [ ]:
# Install Chrome/Chromium for Remotion
!apt-get install -y chromium-browser chromium-chromedriver

# Set Chrome path for Remotion
import os
os.environ['REMOTION_BROWSER_EXECUTABLE'] = '/usr/bin/chromium-browser'


## 2. Upload Project Files


In [ ]:
from google.colab import files
import zipfile
import os

# Create project directory
!mkdir -p /content/video-gen

print("Please upload your project ZIP file (or use git clone)")
print("Option 1: Upload a ZIP file containing your project")
print("Option 2: Use git clone if your project is in a repository")

# Option: Git clone (if project is in a repo)
# !git clone <your-repo-url> /content/video-gen


In [ ]:
# If you uploaded a ZIP file, unzip it
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content/video-gen')
#         print(f"Extracted {filename}")

# Change to project directory
os.chdir('/content/video-gen')
print(f"Current directory: {os.getcwd()}")


## 3. Install Project Dependencies


In [ ]:
# Install npm dependencies
!npm install

print("Dependencies installed!")


## 4. Upload Video Data (JSON Plan)


In [ ]:
# Upload your video plan JSON file
print("Upload your video plan JSON file:")
uploaded = files.upload()

# Find the uploaded JSON file
json_file = None
for filename in uploaded.keys():
    if filename.endswith('.json'):
        json_file = filename
        print(f"Found JSON file: {json_file}")
        break

if not json_file:
    print("No JSON file found. Please upload a video plan JSON file.")


In [ ]:
# Upload assets if needed (images, audio files, etc.)
print("Upload assets folder (optional - if your plan references local assets):")
# assets_uploaded = files.upload()
# You can also use a ZIP file for assets

# Create assets directory structure
!mkdir -p public/assets/gemini-images
!mkdir -p public/assets/voiceovers
!mkdir -p public/assets/vectorized


## 5. Configure Environment


In [ ]:
# Set environment variables for Colab
import os

# Remotion configuration
os.environ['REMOTION_BROWSER_EXECUTABLE'] = '/usr/bin/chromium-browser'
os.environ['REMOTION_BROWSER_TIMEOUT'] = '120000'  # 2 minutes

# FFmpeg configuration
os.environ['FFMPEG_BINARY'] = '/usr/bin/ffmpeg'
os.environ['FFPROBE_BINARY'] = '/usr/bin/ffprobe'

# Optional: Set API keys if needed
# os.environ['GEMINI_API_KEY'] = 'your-api-key'
# os.environ['DEEPGRAM_API_KEY'] = 'your-api-key'

print("Environment configured!")


## 6. Render Video


In [ ]:
# Option 1: Use API endpoint (Recommended)
# Set your API server URL
API_BASE_URL = "http://localhost:3000"  # Change this to your server URL
# For Colab, you might need to use ngrok or a public URL

# Option 2: Direct rendering (if you uploaded files manually)
# Create a rendering script
render_script = f'''
import {{ renderStoryboardVideo }} from './server/services/remotion-ai-renderer';
import {{ readFileSync }} from 'fs';
import {{ join }} from 'path';

async function render() {{
  try {{
    // Load the video plan
    const planPath = join(process.cwd(), '{json_file}');
    const plan = JSON.parse(readFileSync(planPath, 'utf-8'));
    
    console.log('Starting video render...');
    const outputPath = await renderStoryboardVideo(plan);
    
    console.log('Render complete!');
    console.log('Output:', outputPath);
  }} catch (error) {{
    console.error('Render failed:', error);
    process.exit(1);
  }}
}}

render();
'''

# Write script to file
with open('colab-render.ts', 'w') as f:
    f.write(render_script)

print("Render script created!")
print(f"\nAPI Mode: Set API_BASE_URL to your server URL to use API endpoints")
print(f"Direct Mode: Use the render script if you uploaded files manually")


In [ ]:
# Choose rendering method:

# Method 1: Use API (Poll for pending jobs and process them)
import requests
import time
import json

if 'API_BASE_URL' in globals() and API_BASE_URL and API_BASE_URL != "http://localhost:3000":
    print("Using API mode...")
    
    # Poll for pending jobs
    try:
        response = requests.get(f"{API_BASE_URL}/api/colab/jobs/pending")
        if response.status_code == 200:
            pending_jobs = response.json().get('jobs', [])
            print(f"Found {len(pending_jobs)} pending jobs")
            
            for job in pending_jobs:
                job_id = job['jobId']
                plan_url = job['planUrl']
                callback_url = f"{API_BASE_URL}/api/colab/callback/{job_id}"
                
                print(f"\nProcessing job {job_id}...")
                
                # Download plan
                plan_response = requests.get(plan_url)
                plan = plan_response.json()
                
                # Update status to processing
                requests.post(callback_url, json={'status': 'processing', 'startedAt': True})
                
                # Save plan to file
                plan_file = f'{job_id}-plan.json'
                with open(plan_file, 'w') as f:
                    json.dump(plan, f)
                
                # Create render script
                render_script = f'''import {{ renderStoryboardVideo }} from './server/services/remotion-ai-renderer';
import {{ readFileSync }} from 'fs';
import axios from 'axios';

async function render() {{
  try {{
    const plan = JSON.parse(readFileSync('{plan_file}', 'utf-8'));
    console.log('Starting video render for job {job_id}...');
    const outputPath = await renderStoryboardVideo(plan);
    console.log('Render complete! Output:', outputPath);
    
    await axios.post('{callback_url}', {{
      status: 'completed',
      outputPath: outputPath
    }});
    
    console.log('Job completed and callback sent!');
  }} catch (error) {{
    console.error('Render failed:', error);
    await axios.post('{callback_url}', {{
      status: 'failed',
      error: error.message
    }});
    process.exit(1);
  }}
}}

render();'''
                
                script_file = f'colab-render-{job_id}.ts'
                with open(script_file, 'w') as f:
                    f.write(render_script)
                
                # Run render
                get_ipython().system(f'npx ts-node {script_file}')
        else:
            print("No API server available, using direct rendering...")
            get_ipython().system('npx ts-node colab-render.ts')
    except Exception as e:
        print(f"API mode failed: {e}")
        print("Falling back to direct rendering...")
        get_ipython().system('npx ts-node colab-render.ts')
else:
    # Method 2: Direct rendering (manual upload)
    print("Using direct rendering mode...")
    get_ipython().system('npx ts-node colab-render.ts')


## 7. Download Output Video


In [ ]:
# Find the output video file
import glob

output_files = glob.glob('output/*.mp4')
if output_files:
    latest_output = max(output_files, key=os.path.getctime)
    print(f"Found output video: {latest_output}")
    
    # Download the file
    files.download(latest_output)
    print("Download started!")
else:
    print("No output video found. Check the render logs above for errors.")
